In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
import pandas as pd
import time

In [2]:
chromedriver_location = "chrome_driver/chromedriver"

In [3]:
# xPaths to each required search element in the field. 
district_drop_down = '//*[@id="courtId"]'
date_field = '//*[@id="inputVO.startDate"]'
#seven_day_search_radio = '//*[@id="sevenDayRadio"]'
search_by_attorney_radio = '//*[@id="searchAttorneyRadio"]'
mobar_input = '//*[@id="inputVO.mobarNumber"]'
submit_button = '//*[@id="findButton"]'

In [11]:
def slice_per(source, step):
    return [source[i::step] for i in range(step)]

In [16]:
# create an instance of the webdriver and load the inital calendar search page
driver = webdriver.Chrome(chromedriver_location)
driver.get('https://www.courts.mo.gov/casenet/cases/calendarSearch.do')

# store the district dropdown in an object using Select
courtDistrictDrp = Select(driver.find_element_by_xpath(district_drop_down))
courtDistrictDrp.select_by_value("CT11") # 11th District Saint Charles

# date
driver.find_element_by_xpath(date_field).send_keys("8/10/2021")
# 7 day search results
#driver.find_element_by_xpath(seven_day_search_radio).click()
# search by attorney
driver.find_element_by_xpath(search_by_attorney_radio).click()
# MOBAR number field 
driver.find_element_by_xpath(mobar_input).send_keys("71855")

# without the pause, the inputs are too fast for casenet to handle
time.sleep(1) 

# submit 
driver.find_element_by_xpath(submit_button).click()


In [17]:
i=1
data = []
raw_text = []

# get current date
current_date = driver.find_element_by_xpath('/html/body/table/tbody/tr[2]/td/table/tbody/tr[2]/td/table/tbody/tr[1]/td/table/tbody/tr[1]/td[2]').text

# collect information from the table
while driver.find_elements_by_xpath(f'//td[@class="td{i}"]'):
    data.append(driver.find_elements_by_xpath(f'//td[@class="td{i}"]'))
    i += 1

# parse text into a list
for d in data:
    for i in d:
        raw_text.append(i.text)
        
# slices the data into lists of each feild 
organized_data = slice_per(raw_text, 9)

# create data frame to hold the data
df = pd.DataFrame({
    "date" : current_date,
    "case_num" : organized_data[0],
    "case_style" : organized_data[1],
    "time": organized_data[2],
    "days": organized_data[3],
    "event": organized_data[5],
    "location": organized_data[7],
    "judge": organized_data[8]
})
